In [82]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import calendar

In [83]:
os.chdir('c:\\Users\\tyler\\OneDrive\\Documents\\Python\\NFL')
os.getcwd()

'c:\\Users\\tyler\\OneDrive\\Documents\\Python\\NFL'

# Import 2021 Picks

In [84]:
df = pd.read_csv('backend\\data\\predictions\\2021_nn.csv')
df['date'] = pd.to_datetime(df['date'])
df['y'] = np.where(df['y'], df['home'], df['away'])
df['y_pred'] = np.where(df['y_pred'], df['home'], df['away'])
df.dropna(inplace=True)
df.shape

(126, 25)

In [116]:
week = df[df['week'] == 22]
for index, row in week.iterrows():
    ml_a = '+{odds}'.format(odds=row['ml_a']) if row['ml_a'] > 0 else row['ml_a']
    ml_h = '+{odds}'.format(odds=row['ml_h']) if row['ml_h'] > 0 else row['ml_h']
    win = 'W' if row['y_pred'] == row['y'] else 'L'
    print(f"{row['away']} ({ml_a}) @ {row['home']} ({ml_h})")
    print(f"\t {row['y_pred']} **{row['risk_unit']}u** {win}")

Los Angeles Rams (-190) @ Cincinnati Bengals (+160)
	 Los Angeles Rams **1.75u** W


## Betting Analysis 

In [119]:
df['risk_profit'].sum()

-6.801598454063395

In [87]:
# Per Unit
per_unit = df.groupby(['risk_unit']).aggregate({'risk_profit': 'sum', 'risk_correct': ['sum', 'count']})
per_unit['perc'] = (per_unit[('risk_correct', 'sum')] / per_unit[('risk_correct', 'count')] * 100).apply(round)
per_unit

risk_profit risk_correct       perc
                  sum          sum count     
risk_unit                                    
1.00         0.515841           11    14   79
1.25         3.505531           14    18   78
1.50        -1.289734           21    28   75
1.75        -1.665316           19    29   66
2.00        -7.867920           21    37   57

In [89]:
# Per Fav
# Per Unit
per_fav = df.groupby(['pick_fav']).aggregate({'risk_profit': 'sum', 'risk_correct': ['sum', 'count']})
per_fav['perc'] = (per_fav[('risk_correct', 'sum')] / per_fav[('risk_correct', 'count')] * 100).apply(round)
per_fav

risk_profit risk_correct       perc
                 sum          sum count     
pick_fav                                    
0           8.725000           10    18   56
1         -15.526598           76   108   70

In [90]:
# Per Month
df['month'] = df['date'].dt.month
df['month'] = df['month'].apply(lambda x: calendar.month_abbr[x])
per_month = df.groupby(['month']).aggregate({'risk_profit': 'sum', 'risk_correct': ['sum', 'count']})
per_month['perc'] = (per_month[('risk_correct', 'sum')] / per_month[('risk_correct', 'count')] * 100).apply(round)
per_month.sort_values(by=[('risk_profit', 'sum'), ], ascending=False)

risk_profit risk_correct       perc
              sum          sum count     
month                                    
Dec      7.082795           26    34   76
Feb      0.921053            1     1  100
Sep      0.500000            1     1  100
Jan     -1.539153           17    22   77
Oct     -3.711736           22    35   63
Nov    -10.054557           19    33   58

In [95]:
# Per Week
per_week = df.groupby(['week']).aggregate({'risk_profit': 'sum', 'risk_correct': ['sum', 'count']})
per_week['perc'] = (per_week[('risk_correct', 'sum')] / per_week[('risk_correct', 'count')] * 100).apply(round)
per_week

risk_profit risk_correct       perc
             sum          sum count     
week                                    
4       4.212500            4     5   80
5      -4.001601            5     9   56
6       0.003030            5     7   71
7      -0.709757            7    10   70
8      -2.430195            3     6   50
9      -5.640245            5    10   50
10     -6.285169            3     8   38
11      2.389472            7     9   78
12     -0.804329            3     5   60
13      2.563154            7     9   78
14      4.240196            6     6  100
15      0.023973            7    11   64
16      0.255472            6     8   75
17      1.241294           11    13   85
18     -3.451645            2     4   50
19      0.671198            4     5   80
22      0.921053            1     1  100

In [92]:
df[df['week'] == 10][['date', 'home', 'ml_h', 'away', 'ml_a', 'y', 'y_pred', 'pick_diff', 'risk_unit', 'risk_profit']]

,date,home,ml_h,away,ml_a,y,y_pred,pick_diff,risk_unit,risk_profit
86,2021-11-11,Miami Dolphins,310,Baltimore Ravens,-410,Miami Dolphins,Baltimore Ravens,0.195174,1.75,-1.750000
87,2021-11-14,Arizona Cardinals,-310,Carolina Panthers,245,Carolina Panthers,Arizona Cardinals,0.030071,1.00,-1.000000
88,2021-11-14,Dallas Cowboys,-335,Atlanta Falcons,260,Dallas Cowboys,Dallas Cowboys,0.223595,2.00,0.597015
91,2021-11-14,Indianapolis Colts,-450,Jacksonville Jaguars,340,Indianapolis Colts,Indianapolis Colts,0.117002,1.50,0.333333
92,2021-11-14,Las Vegas Raiders,125,Kansas City Chiefs,-145,Kansas City Chiefs,Kansas City Chiefs,0.105153,1.50,1.034483
93,2021-11-14,Los Angeles Chargers,-180,Minnesota Vikings,155,Minnesota Vikings,Los Angeles Chargers,0.194848,1.75,-1.750000
98,2021-11-14,Washington Football Team,340,Tampa Bay Buccaneers,-450,Washington Football Team,Tampa Bay Buccaneers,0.171814,1.75,-1.750000
99,2021-11-15,San Francisco 49ers,160,Los Angeles Rams,-190,San Francisco 49ers,Los Angeles Rams,0.248221,2.00,-2.000000


In [93]:
# Per Team
per_team = df.groupby(['y_pred']).aggregate({'risk_profit': 'sum', 'risk_correct': ['sum', 'count']})
per_team['perc'] = (per_team[('risk_correct', 'sum')] / per_team[('risk_correct', 'count')] * 100).apply(round)
per_team.sort_values(by=[('risk_profit', 'sum'), ], ascending=False)

risk_profit risk_correct       perc
                                 sum          sum count     
y_pred                                                      
Houston Texans              4.300000            1     1  100
Minnesota Vikings           2.989560            3     3  100
San Francisco 49ers         2.867588            5     5  100
New England Patriots        2.414612            4     4  100
Jacksonville Jaguars        2.200000            1     1  100
Kansas City Chiefs          1.855868           10    12   83
Washington Football Team    1.837500            1     1  100
Chicago Bears               1.729032            2     2  100
New York Jets               1.587500            1     2   50
New York Giants             1.562500            1     1  100
Green Bay Packers           1.317176            2     2  100
Atlanta Falcons             1.200000            1     1  100
Pittsburgh Steelers         0.491803            1     1  100
Miami Dolphins              0.480769            1     1  100
Las Vegas Raiders          -0.095238            1     2   50
Tampa Bay Buccaneers       -0.150588           10    13   77
Los Angeles Rams           -0.217602            6     8   75
Los Angeles Chargers       -0.320149            4     6   67
Indianapolis Colts         -0.560606            3     4   75
Denver Broncos             -1.083333            1     2   50
Philadelphia Eagles        -1.603296            4     6   67
Cincinnati Bengals         -1.803571            2     4   50
Buffalo Bills              -1.832341            5     8   62
Tennessee Titans           -2.643791            2     4   50
Seattle Seahawks           -2.693074            2     4   50
Cleveland Browns           -3.019231            1     3   33
Dallas Cowboys             -3.483452            4     7   57
Baltimore Ravens           -3.506218            4     8   50
Arizona Cardinals          -4.458333            1     4   25
New Orleans Saints         -6.164683            2     6   33